In [2]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Problem 1 Version 1

In [3]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.8
m.cap = 15
m.dispCost = 1.2

# Define implicit variables
m.MarkPrice = Var(m.ITEMS, domain = NonNegativeReals)
m.Demand = Var(m.ITEMS, domain = NonNegativeReals)
m.Revenue = Var(m.ITEMS, domain = NonNegativeReals)
m.FinalInv = Var(m.ITEMS, domain = NonNegativeReals)

def Define_MarkPrice_Rule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i] == m.MarkPrice[i]

def Define_Demand_Rule(m, i):
    return m.MarkPct[i] * m.initInv[i] == m.Demand[i]

def Define_Revenue_Rule(m, i):
    return m.MarkPrice[i] * m.Demand[i] == m.Revenue[i]

def Define_FinalInv(m, i):
    return m.initInv[i] - m.Demand[i] == m.FinalInv[i]

def Max_Discount_Rule(m, i):
    return m.MarkPct[i] <= m.maxPct

def Leftover_Inv_Rule(m, i):
    return m.FinalInv[i] <= m.cap

m.Define_MarkPrice = Constraint(m.ITEMS, rule = Define_MarkPrice_Rule)
m.Define_Demand = Constraint(m.ITEMS, rule = Define_Demand_Rule)
m.Define_Revenue = Constraint(m.ITEMS, rule = Define_Revenue_Rule)
m.Define_FinalInv = Constraint(m.ITEMS, rule = Define_FinalInv)
m.Max_Discount_Rule = Constraint(m.ITEMS, rule = Max_Discount_Rule)
m.Leftover_Inv_Rule = Constraint(m.ITEMS, rule = Leftover_Inv_Rule)

def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [4]:
# Won't work, error = Model constraint (Define_Revenue[A]) contains nonlinear terms that cannot be written to LP format
# opt = SolverFactory('glpk')
# results = opt.solve(m, tee = True)

In [5]:
from pyomo.contrib import appsi
SOLVER = appsi.solvers.Ipopt()
SOLVER.available()

<Availability.FullLicense: 1>

In [6]:
results = SOLVER.solve(m)

In [7]:
m.display()

Model unknown

  Variables:
    MarkPct : Size=3, Index=ITEMS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          A :     0 : 0.5279069767121749 :     1 : False : False :  Reals
          B :     0 : 0.5203389831118836 :     1 : False : False :  Reals
          C :     0 : 0.6341463378136317 :     1 : False : False :  Reals
    MarkPrice : Size=3, Index=ITEMS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          A :     0 :  10.15000000068824 :  None : False : False : NonNegativeReals
          B :     0 : 14.149999998199432 :  None : False : False : NonNegativeReals
          C :     0 : 11.158536696684234 :  None : False : False : NonNegativeReals
    Demand : Size=3, Index=ITEMS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          A :     0 :  5.279069767121749 :  None : False : False : NonNegativeReals
          B :     0 : 15.610169493356507 :  None : False : False : NonNegativeReals
     

### Trying using params

In [38]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.8
m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPct[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m, i):
    return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [39]:
results = SOLVER.solve(m)
m.display()

Model unknown

  Variables:
    MarkPct : Size=3, Index=ITEMS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          A :     0 : 0.8000000099105196 :     1 : False : False :  Reals
          B :     0 : 0.8000000099701684 :     1 : False : False :  Reals
          C :     0 : 0.8000000099781712 :     1 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 32.40000226048276

  Constraints:
    MaxDiscount : Size=3
        Key : Lower : Body               : Upper
          A :  None : 0.8000000099105196 :   0.8
          B :  None : 0.8000000099701684 :   0.8
          C :  None : 0.8000000099781712 :   0.8
    LeftOverInvRule : Size=3
        Key : Lower : Body               : Upper
          A :  None : 1.9999999008948048 :  15.0
          B :  None :  5.999999700894946 :  15.0
          C :  None :  8.199999590894983 :  15.0
